## Chapter 11: Testing Aspire Applications

Testing distributed applications is notoriously difficult. With multiple services, databases, message queues, and external dependencies, traditional unit tests only cover isolated pieces. Integration tests often require complex setup and teardown, and are slow and brittle. .NET Aspire changes this by providing a testing paradigm that leverages the AppHost itself. With the `Aspire.Hosting.Testing` package, you can spin up your entire application (or a subset) in a test, interact with it, and assert outcomes—all in a fast, repeatable way.

In this chapter, you’ll learn:

- The challenges of testing distributed systems and how Aspire addresses them.
- How to write integration tests that start the AppHost and run against real (containerized) dependencies.
- Using `DistributedApplicationTestingBuilder` to create a test fixture.
- Writing tests for HTTP endpoints, health checks, and messaging.
- Testing resilience and failure scenarios.
- Best practices for test organization and cleanup.

By the end, you’ll be able to write robust integration tests that give you confidence in your Aspire application.

---

### 11.1 The Challenge of Testing Distributed Systems

Consider our e‑commerce application. It consists of:

- A web frontend (Blazor)
- An API service (ASP.NET Core)
- A PostgreSQL database
- A RabbitMQ message broker
- A worker service that processes messages
- Azure Blob Storage (or emulator)

Testing a single component in isolation (e.g., the API with a mocked database) can verify business logic, but it won’t catch issues like:

- Incorrect connection strings or configuration injection.
- Service discovery failures (e.g., the frontend can’t find the API).
- Message serialization mismatches between publisher and consumer.
- Database migration errors.
- Resilience policies not working as expected.

End‑to‑end tests that run the whole system are essential, but they’re often slow and hard to automate. Aspire’s testing support allows you to run the AppHost in a test—spinning up only what you need, with fast container startup and deterministic cleanup.

---

### 11.2 Introducing Aspire.Hosting.Testing

The `Aspire.Hosting.Testing` package provides:

- `DistributedApplicationTestingBuilder`: a builder for creating a test‑specific AppHost.
- `DistributedApplication` extensions for test interaction (e.g., `CreateHttpClient`, `GetConnectionString`).
- Automatic resource cleanup after tests.

The key idea: you create an instance of your AppHost **in‑process** within the test runner. The AppHost starts all the necessary containers and processes, just as it would when you run `dotnet run`. Your test can then make HTTP calls to services, send messages, and assert conditions. After the test, everything is torn down.

Because containers are reused across tests (if you use a shared fixture), the overhead is minimal.

---

### 11.3 Setting Up a Test Project

Let’s create a new test project in our `MyAspireApp` solution. We’ll use xUnit, but you can use any test framework.

```bash
dotnet new xunit -n MyAspireApp.Tests
dotnet sln add MyAspireApp.Tests
cd MyAspireApp.Tests
```

Add the necessary packages:

```bash
dotnet add package Aspire.Hosting.Testing
dotnet add package Microsoft.AspNetCore.Mvc.Testing  # optional, for HttpClient helpers
dotnet add coverlet.collector  # for code coverage
```

Also add a project reference to the AppHost project (since we need to reference the AppHost’s `Program` class to build it):

```bash
dotnet add reference ../MyAspireApp.AppHost
```

Now we’re ready to write tests.

---

### 11.4 Writing Your First Integration Test

We’ll write a simple test that verifies the API’s weather forecast endpoint returns a successful response.

#### 11.4.1 Creating a Test Fixture

First, create a test fixture that starts the AppHost once per test class (or once per test run). xUnit provides `IClassFixture` for shared fixture instances.

Create a class `AspireAppFixture.cs`:

```csharp
using Aspire.Hosting.Testing;
using Microsoft.Extensions.Hosting;

namespace MyAspireApp.Tests;

public class AspireAppFixture : IAsyncLifetime
{
    private DistributedApplication? _app;

    public DistributedApplication App => _app ?? throw new InvalidOperationException("App not started");

    public async Task InitializeAsync()
    {
        var builder = await DistributedApplicationTestingBuilder.CreateAsync(typeof(Projects.MyAspireApp_AppHost));
        _app = await builder.BuildAsync();
        await _app.StartAsync();
    }

    public async Task DisposeAsync()
    {
        if (_app != null)
        {
            await _app.DisposeAsync();
        }
    }
}
```

**Explanation**:

- `DistributedApplicationTestingBuilder.CreateAsync` takes the type of the AppHost’s project (generated by the Projects class) and creates a builder configured for testing. It automatically sets the environment to "Testing" and disables dashboard (you can override).
- `builder.BuildAsync()` creates the `DistributedApplication` instance.
- `_app.StartAsync()` starts all resources. This is asynchronous and will wait until all resources are running (or fail).
- The fixture implements `IAsyncLifetime` so xUnit will call `InitializeAsync` before tests and `DisposeAsync` after.

#### 11.4.2 Writing a Test That Uses the Fixture

Now create a test class that uses the fixture:

```csharp
using Aspire.Hosting.Testing;
using System.Net.Http.Json;

namespace MyAspireApp.Tests;

public class ApiTests : IClassFixture<AspireAppFixture>
{
    private readonly AspireAppFixture _fixture;

    public ApiTests(AspireAppFixture fixture)
    {
        _fixture = fixture;
    }

    [Fact]
    public async Task GetWeatherForecast_ReturnsSuccess()
    {
        // Arrange
        var httpClient = _fixture.App.CreateHttpClient("apiservice");

        // Act
        var response = await httpClient.GetAsync("/weatherforecast");

        // Assert
        response.EnsureSuccessStatusCode();
        var forecasts = await response.Content.ReadFromJsonAsync<WeatherForecast[]>();
        Assert.NotNull(forecasts);
        Assert.NotEmpty(forecasts);
    }
}
```

**Explanation**:

- `_fixture.App.CreateHttpClient("apiservice")` creates an `HttpClient` configured to talk to the `apiservice` resource. It uses service discovery internally—the client’s base address is set to `http://apiservice`, and it resolves to the actual endpoint of the API service instance.
- We then make a GET request and assert.

Run the test. It will start the AppHost (PostgreSQL, RabbitMQ, etc.), run the API service, and make the request. The first run may be slow due to container pulls, but subsequent runs are faster.

#### 11.4.3 Accessing Connection Strings

You might need to directly interact with a database or queue. The `DistributedApplication` provides a `GetConnectionString` method:

```csharp
var connectionString = _fixture.App.GetConnectionString("productsdb");
```

You can then use this to create a database connection and run queries.

---

### 11.5 Testing Health Checks

Aspire components automatically add health checks. You can test that the health endpoint returns healthy.

```csharp
[Fact]
public async Task HealthCheck_ReturnsHealthy()
{
    var httpClient = _fixture.App.CreateHttpClient("apiservice");
    var response = await httpClient.GetAsync("/health");
    response.EnsureSuccessStatusCode();
    var content = await response.Content.ReadAsStringAsync();
    Assert.Contains("Healthy", content);
}
```

You can also test the readiness endpoint if you mapped it.

---

### 11.6 Testing Messaging

To test messaging, you need to interact with the message broker. Using the connection string, you can create a RabbitMQ client and verify messages are published and consumed.

First, add the RabbitMQ client package to the test project:

```bash
dotnet add package Aspire.RabbitMQ.Client
```

Then write a test:

```csharp
[Fact]
public async Task OrderCreated_PublishesMessage()
{
    // Arrange: create an order via API
    var apiClient = _fixture.App.CreateHttpClient("apiservice");
    var orderRequest = new CreateOrderRequest("Test Customer", new List<OrderItemDto>
    {
        new(ProductId: 1, Quantity: 2)
    });

    var response = await apiClient.PostAsJsonAsync("/orders", orderRequest);
    response.EnsureSuccessStatusCode();
    var orderResult = await response.Content.ReadFromJsonAsync<OrderResponse>();
    Assert.NotNull(orderResult);

    // Wait a bit for message processing
    await Task.Delay(2000);

    // Assert: check RabbitMQ queue
    var connectionString = _fixture.App.GetConnectionString("messaging");
    var connectionFactory = new ConnectionFactory
    {
        Uri = new Uri(connectionString)
    };
    using var connection = await connectionFactory.CreateConnectionAsync();
    using var channel = await connection.CreateChannelAsync();

    // Declare queue passively to check message count
    var queueDeclareOk = await channel.QueueDeclarePassiveAsync("order-created-queue");
    Assert.True(queueDeclareOk.MessageCount > 0);

    // Consume one message and verify content
    var consumer = new AsyncEventingBasicConsumer(channel);
    var tcs = new TaskCompletionSource<OrderCreatedEvent>();
    consumer.ReceivedAsync += async (model, ea) =>
    {
        var body = ea.Body.ToArray();
        var message = JsonSerializer.Deserialize<OrderCreatedEvent>(Encoding.UTF8.GetString(body));
        tcs.SetResult(message!);
        await channel.BasicAckAsync(ea.DeliveryTag, false);
    };
    await channel.BasicConsumeAsync("order-created-queue", autoAck: false, consumer: consumer);

    var receivedEvent = await tcs.Task.WaitAsync(TimeSpan.FromSeconds(5));
    Assert.Equal(orderResult.OrderId, receivedEvent.OrderId);
}
```

This test publishes an order, then connects to RabbitMQ, checks the queue, and consumes the message. It verifies that the message content matches the order ID.

**Note**: In a real test, you’d want to avoid `Task.Delay` and instead poll or use a more robust wait mechanism. You can use `await WaitForQueueMessageCountAsync` helper (you'd need to implement) or retry logic.

---

### 11.7 Testing Database State

You can also verify that the order was saved in the database:

```csharp
[Fact]
public async Task OrderCreated_SavesToDatabase()
{
    // Arrange
    var apiClient = _fixture.App.CreateHttpClient("apiservice");
    var orderRequest = new CreateOrderRequest("Test Customer", new List<OrderItemDto>
    {
        new(ProductId: 1, Quantity: 2)
    });

    // Act
    var response = await apiClient.PostAsJsonAsync("/orders", orderRequest);
    response.EnsureSuccessStatusCode();
    var orderResult = await response.Content.ReadFromJsonAsync<OrderResponse>();

    // Assert: check database
    var connectionString = _fixture.App.GetConnectionString("productsdb");
    await using var dataSource = NpgsqlDataSource.Create(connectionString);
    await using var conn = await dataSource.OpenConnectionAsync();
    var cmd = new NpgsqlCommand("SELECT COUNT(*) FROM Orders WHERE Id = @id", conn);
    cmd.Parameters.AddWithValue("@id", orderResult.OrderId);
    var count = (long)await cmd.ExecuteScalarAsync();
    Assert.Equal(1, count);
}
```

---

### 11.8 Testing Resilience and Failure Scenarios

One of the biggest benefits of testing with the real AppHost is the ability to simulate failures. For example, you can stop a container and see how your application behaves.

Aspire’s testing API doesn’t directly expose container controls, but you can use Docker SDK or simply rely on the fact that resources are processes; you could send a stop signal. However, a simpler approach is to leverage the `IResourceBuilder` and annotations to modify behavior for testing, or to use the `WithAnnotation` to inject a test flag that the service recognizes.

For resilience testing, you might want to temporarily break a dependency. One way is to use a **test double** container. For example, you could replace the PostgreSQL container with a pre‑configured one that fails on certain queries. That’s complex.

A more practical approach: rely on the built‑in resilience and test that your application handles transient failures gracefully. For instance, you can stop the RabbitMQ container mid‑test (using Docker commands) and then restart it, and assert that messages are eventually processed.

To stop a container, you need its container ID. Aspire’s resources don’t expose that directly, but you can get it from the Docker API. A simpler way is to use `await _fixture.App.StopResourceAsync("messaging")` – but is there such a method? Not in the current API. However, you can get the container’s name from the dashboard and use `docker stop` from the test.

Here’s an example using the `Docker.DotNet` SDK:

```csharp
// Install Docker.DotNet
// Get container ID from resource name (requires custom logic)
```

But to keep it simple, we’ll skip full implementation. Instead, we can test resilience by making the API service throw exceptions temporarily, but that requires modifying the service.

A better approach: use **fault injection** via Polly. You could configure a custom resilience handler that, in test mode, simulates failures. But that’s more of an advanced topic.

For the purpose of this chapter, we’ll assume we can test resilience by ensuring that retries happen when a dependency is briefly unavailable.

---

### 11.9 Using Test Hooks and Configuration

You may want to modify the AppHost for testing—for example, to use a different database container, or to disable certain services. The `DistributedApplicationTestingBuilder` allows you to customize the AppHost before building.

```csharp
var builder = await DistributedApplicationTestingBuilder.CreateAsync(typeof(Projects.MyAspireApp_AppHost));

// Override a resource's configuration
builder.Services.Configure<PostgresDatabaseResource>(postgres =>
{
    // Change something? Not directly.
});

// Or add a test-specific environment variable to a project
var apiService = builder.Resources.OfType<ProjectResource>().First(r => r.Name == "apiservice");
apiService.Annotations.Add(new EnvironmentCallbackAnnotation(context =>
{
    context.EnvironmentVariables["TEST_MODE"] = "true";
}));

_app = await builder.BuildAsync();
```

This allows you to inject test‑specific behavior into your services.

---

### 11.10 Organizing Tests for Performance

Starting the full AppHost for every test class can be slow. You can use **collection fixtures** in xUnit to share a single AppHost instance across multiple test classes. This is efficient but means tests run serially and may interfere with each other (e.g., if they modify shared state). You can also use **test containers** to isolate tests by using a fresh database per test, but that’s more complex.

A common pattern:

- Use a fixture that starts the AppHost once per test run (collection fixture).
- For tests that modify state, use unique identifiers (e.g., random product IDs) to avoid collisions.
- Or, use transaction rollbacks at the database level.

---

### 11.11 Hands‑on: Write a Test for the Order Workflow

Let’s write a comprehensive test that verifies the entire order workflow: API receives order, saves to database, publishes message, and worker processes it.

We’ll need to:

1. Create an order via API.
2. Wait for the worker to process (maybe by checking logs or a database flag).
3. Assert that the order was processed (e.g., a processed flag in database, or a message in a dead‑letter queue).

For simplicity, we’ll assume the worker updates the order with a `ProcessedAt` timestamp.

First, modify the worker to update the database when it processes an order. In the worker’s `OrderConsumer`, after logging, add a database update:

```csharp
// In OrderConsumer, after successful processing
using var dataSource = NpgsqlDataSource.Create(connectionString); // get connection string from config
using var conn = await dataSource.OpenConnectionAsync();
await conn.ExecuteAsync("UPDATE Orders SET ProcessedAt = @now WHERE Id = @orderId", new { now = DateTime.UtcNow, orderId = orderEvent.OrderId });
```

You’ll need to inject the database connection string into the worker. Add a reference to the database in the AppHost for the worker:

```csharp
workerService.WithReference(productsDb);
```

Then in the worker’s `Program.cs`, register `NpgsqlDataSource` using the connection string (similar to API). But note: the worker is a `BackgroundService`, not a web app. You can still use `builder.AddNpgsqlDataSource("productsdb")` because the worker uses the generic host, which supports the same component.

Now the test:

```csharp
[Fact]
public async Task FullOrderWorkflow_Succeeds()
{
    // Arrange
    var apiClient = _fixture.App.CreateHttpClient("apiservice");
    var orderRequest = new CreateOrderRequest("Integration Test", new List<OrderItemDto>
    {
        new(ProductId: 1, Quantity: 1)
    });

    // Act
    var response = await apiClient.PostAsJsonAsync("/orders", orderRequest);
    response.EnsureSuccessStatusCode();
    var orderResult = await response.Content.ReadFromJsonAsync<OrderResponse>();

    // Wait for worker to process (poll database)
    var connectionString = _fixture.App.GetConnectionString("productsdb");
    await using var dataSource = NpgsqlDataSource.Create(connectionString);
    DateTime? processedAt = null;
    for (int i = 0; i < 10; i++)
    {
        await using var conn = await dataSource.OpenConnectionAsync();
        processedAt = await conn.ExecuteScalarAsync<DateTime?>("SELECT ProcessedAt FROM Orders WHERE Id = @id", new { id = orderResult.OrderId });
        if (processedAt != null) break;
        await Task.Delay(1000);
    }

    // Assert
    Assert.NotNull(processedAt);
}
```

This test polls the database up to 10 seconds for the `ProcessedAt` field to be set. If it’s set, the worker processed the order.

---

### 11.12 Summary

In this chapter, you learned how to write integration tests for Aspire applications using the `Aspire.Hosting.Testing` package. Key takeaways:

- The testing package allows you to spin up the full AppHost in a test, including all containers and services.
- You can create HTTP clients that automatically use service discovery to talk to services.
- You can access connection strings to interact directly with databases and message queues.
- Tests can verify health checks, API endpoints, messaging, and database state.
- You can organize tests with fixtures to share the AppHost instance, improving performance.
- While failure testing is more complex, you can still validate resilience by observing retries and eventual consistency.

With these techniques, you can build a comprehensive test suite that gives you confidence in your distributed system. In the next chapter, we’ll move on to **Security from the Start**, where you’ll learn how to secure your Aspire applications with authentication, authorization, and secure communication.

---

**Exercises**

1. Write a test that verifies the product upload endpoint works and that the blob is stored in Azurite. Use `BlobServiceClient` with the connection string.
2. Write a test that stops the RabbitMQ container (using Docker commands) while an order is being processed, and then restarts it, verifying that the message is eventually consumed (use a dead‑letter queue if possible).
3. Create a test that verifies the API’s health check fails when the database is stopped. Use `IResourceBuilder` to stop the database container (you may need to use Docker SDK).
4. Implement a test fixture that starts a subset of resources (e.g., only API and database) for faster tests.

In Chapter 12, we’ll explore **Security from the Start**, adding authentication and authorization to your Aspire application.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='10. advanced_service_discovery_and_resiliency.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='12. security_from_the_start.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
